## 明日の天気予報をLINEで通知するプログラム
### LINE Notify + Webスクレイピングを使用する

- API:Application Programming Interface→「鍵付きの扉」のイメージ  
発行してもらった鍵を使い、プログラムから指定されたURLにアクセスすると、サービスが使える

In [2]:
import requests

requests-> HTTP通信用のライブラリ  

In [3]:
import secret

token = secret.line_token
url = secret.line_url

GitHubに載せるための措置。secretに個人情報を入れて`.gitignore`で指定し、公開しないようにする。

In [4]:
auth = {"Authorization": "Bearer " + token}
content = {"message": "Python test. Hello, I`m Python"}

In [5]:
requests.post(url, headers=auth, data=content)

<Response [200]>

=> 成功。LINENotifyから通知  
続いて、Webスクレイピングして情報を取得する作業

In [6]:
from bs4 import BeautifulSoup

BeautifulSoup-> HTMLの情報を扱う。情報を取得する  
Selenium→ブラウザ上での操作を伴う

In [7]:
# weather newsから天気予報を取得
weather_url = "https://weathernews.jp/onebox/tenki/osaka/27203/"

In [35]:
response = requests.get(weather_url)

In [36]:
response.text

'<!DOCTYPE html>\n<html lang="ja">\n<!-- 34.7765468, 135.4696758, 0 -->\n\n<head>\n    <meta charset="UTF-8">\n    <title>大阪府豊中市の天気予報(1時間・今日明日・週間) - ウェザーニュース</title>\n    <meta name="viewport"\n        content="width=device-width,initial-scale=1.0,minimum-scale=1.0,maximum-scale=1.0,user-scalable=no">\n    <meta name="apple-itunes-app" content="app-id=302955766">\n    <meta name="viewport" content="width=device-width, initial-scale=1.0" />\n    <meta name="description" content="【予報精度No.1】大阪府豊中市の天気予報を5分毎・1時間毎・今日明日・週間(10日間)で掲載中！今知りたい現地の天気は、ウェザーニュースアプリから届く空の写真で確認。世界最大級の気象情報会社ウェザーニューズの観測ネットワークと独自の予測モデル、AI分析で一番当たる予報をお届けします。" />\n    <meta name="keywords" content="大阪府豊中市,天気,予報,気象,ピンポイント,天気予報,週間天気,リアルタイム,現在,過去,服装,観光,ウェザーニューズ,ウェザーニュース" />\n    <meta name="twitter:card" content="summary_large_image" />\n    <meta name="twitter:site" content="@wni_jp" />\n    <meta name="twitter:creator" content="@wni_jp">\n    <meta name="twitter:image" content="https://weathernews.jp/onebox/img/og_img.jpg">\n 

In [37]:
html = BeautifulSoup(response.text, "html.parser")

In [40]:
html.find_all("div")

[<div class="inner">
 <figure class="wniLogo"><a href="https://weathernews.jp/?fm=header"><img alt="ウェザーニュース" height="40px" src="https://weathernews.jp/onebox/img/header/logo_wni.svg" width="185px"/></a></figure>
 <div class="headerWeatherList" v-cloak="">
 <div class="headerWeatherList__inner">
 <div :class="{'new':index==0, 'exiting':index!=0}" :id="index" class="headerWeatherList__item" v-for="(item,index) in fcst_hist">
 <a :href="item.url + '?fm=header'" class="headerWeatherList__link"></a>
 <figure class="headerWeatherList__icon"><img :src="item.telop" height="38px" width="38px"/></figure>
 <div class="headerWeatherList__content">
 <p class="headerWeatherList__area">{{item.cityname}}</p>
 <div class="headerWeatherList__temp">
 <span class="h">{{item.high}}</span>/<span class="l">{{item.low}}</span>
 <p @click="historyClose(index)" class="headerWeatherList__close"></p>
 </div>
 </div>
 </div>
 </div>
 </div>
 <div class="headerSearchArea">
 <form onsubmit="search_city_input('searc

In [60]:
html.find_all("div", attrs={"class": "day2Table"})

[<div class="day2Table">
 <div class="day2Table__item weather">
 <img class="wIcon" src="//weathernews.jp/onebox/img/wxicon/203.png"/>
 </div>
 <div class="day2Table__item day2Info no-global">
 <p class="title"></p>
 <ul class="list"><li>日の出 07:06</li><li>日の入 17:05</li></ul>
 </div>
 </div>,
 <div class="day2Table day2__temp">
 <div class="day2Table__item">
 <p class="temp__h"><span class="badge">最高</span><span class="text wTable__item">9<span class="unit">℃</span></span><span class="difference"></span></p>
 </div>
 <div class="day2Table__item">
 <p class="temp__l"><span class="badge">最低</span><span class="text wTable__item">2<span class="unit">℃</span></span><span class="difference"></span></p>
 </div>
 </div>,
 <div class="day2Table day2__temp">
 <div class="day2Table__item">
 <p><span class="badge">午前</span><span class="text wTable__item">20<span class="unit">%</span></span><span class="difference"></span></p>
 </div>
 <div class="day2Table__item">
 <p><span class="badge">午後</span><

In [75]:
html.find_all("div", attrs={"class": "day2Table__item weather"})

[<div class="day2Table__item weather">
 <img class="wIcon" src="//weathernews.jp/onebox/img/wxicon/203.png"/>
 </div>,
 <div class="day2Table__item weather">
 <img class="wIcon" src="//weathernews.jp/onebox/img/wxicon/100.png"/>
 </div>]

In [43]:
html.find_all("div", attrs={"class": "day2Table day2__temp"})

[<div class="day2Table day2__temp">
 <div class="day2Table__item">
 <p class="temp__h"><span class="badge">最高</span><span class="text wTable__item">9<span class="unit">℃</span></span><span class="difference"></span></p>
 </div>
 <div class="day2Table__item">
 <p class="temp__l"><span class="badge">最低</span><span class="text wTable__item">2<span class="unit">℃</span></span><span class="difference"></span></p>
 </div>
 </div>,
 <div class="day2Table day2__temp">
 <div class="day2Table__item">
 <p><span class="badge">午前</span><span class="text wTable__item">20<span class="unit">%</span></span><span class="difference"></span></p>
 </div>
 <div class="day2Table__item">
 <p><span class="badge">午後</span><span class="text wTable__item">30<span class="unit">%</span></span><span class="difference"></span></p>
 </div>
 </div>,
 <div class="day2Table day2__temp">
 <div class="day2Table__item">
 <p class="temp__h"><span class="badge">最高</span><span class="text wTable__item">12<span class="unit">℃</

In [100]:
comment = html.find_all("div", attrs={"class": "comment no-ja sp"})[0].text
comment

'\n折りたたみ傘と万全な防寒を\n今日は雲が多い空で、冷たい雨の降ることがあります。外出の際は傘を忘れずに。昼間も日差しが乏しく、寒い一日です。万全な防寒が欠かせません。\n'

In [61]:
forecast = html.find_all("div", attrs={"class": "day2Table day2__temp"})

In [63]:
# print(type(forecast))
# len(forecast) => 4: 今日の気温、降水確率、明日の気温、降水確率
# 天気は画像、100.png=>晴れ、203.png=>曇り
forecast

[<div class="day2Table day2__temp">
 <div class="day2Table__item">
 <p class="temp__h"><span class="badge">最高</span><span class="text wTable__item">9<span class="unit">℃</span></span><span class="difference"></span></p>
 </div>
 <div class="day2Table__item">
 <p class="temp__l"><span class="badge">最低</span><span class="text wTable__item">2<span class="unit">℃</span></span><span class="difference"></span></p>
 </div>
 </div>,
 <div class="day2Table day2__temp">
 <div class="day2Table__item">
 <p><span class="badge">午前</span><span class="text wTable__item">20<span class="unit">%</span></span><span class="difference"></span></p>
 </div>
 <div class="day2Table__item">
 <p><span class="badge">午後</span><span class="text wTable__item">30<span class="unit">%</span></span><span class="difference"></span></p>
 </div>
 </div>,
 <div class="day2Table day2__temp">
 <div class="day2Table__item">
 <p class="temp__h"><span class="badge">最高</span><span class="text wTable__item">12<span class="unit">℃</

In [67]:
today_max_temp = forecast[0].find_all(
    "span", attrs={"class": "text wTable__item"}
)[0].text[:-1]
today_min_temp = forecast[0].find_all(
    "span", attrs={"class": "text wTable__item"}
)[1].text[:-1]

today_am_rainy_per = forecast[1].find_all(
    "span", attrs={"class": "text wTable__item"}
)[0].text.replace("%", "")
today_pm_rainy_per = forecast[1].find_all(
    "span", attrs={"class": "text wTable__item"}
)[1].text.replace("%", "")
# 文字列型を数値型に変換して、ifの条件分岐に用いるために%を省いている

取り出したテキストに`\n`などが含まれていた場合、
```Python
.replace("置換前文字列", "置換後文字列")
```
とする。

In [68]:
print(today_max_temp, today_min_temp, am_rainy_per, pm_rainy_per)

9 2 20 30


In [56]:
type(am_rainy_per)

str

In [101]:
message = f"""\
本日の最高気温は{today_max_temp}度、最低気温は{today_min_temp}度。
降水確率は午前が{today_am_rainy_per}%、午後が{today_pm_rainy_per}%です。{comment}"""
message

'本日の最高気温は9度、最低気温は2度。\n降水確率は午前が20%、午後が30%です。\n折りたたみ傘と万全な防寒を\n今日は雲が多い空で、冷たい雨の降ることがあります。外出の際は傘を忘れずに。昼間も日差しが乏しく、寒い一日です。万全な防寒が欠かせません。\n'

In [102]:
auth = {"Authorization": "Bearer " + token}
content = {"message": message}
requests.post(url, headers=auth, data=content)

<Response [200]>

### このプログラムを定期実行する方法
1. .pyファイルを作成する
2. .pyファイルを定期実行→Windows:タスクスケジューラ

In [2]:
import requests
from bs4 import BeautifulSoup

import secret

token = secret.line_token
url = secret.line_url
weather_url = "https://weathernews.jp/onebox/tenki/osaka/27203/"

response = requests.get(weather_url)
html = BeautifulSoup(response.text, "html.parser")

comment = html.find_all("div", attrs={"class": "comment no-ja sp"})[0].text
forecast = html.find_all("div", attrs={"class": "day2Table day2__temp"})

today_max_temp = forecast[0].find_all(
    "span", attrs={"class": "text wTable__item"}
)[0].text[:-1]
today_min_temp = forecast[0].find_all(
    "span", attrs={"class": "text wTable__item"}
)[1].text[:-1]

today_am_rainy_per = forecast[1].find_all(
    "span", attrs={"class": "text wTable__item"}
)[0].text.replace("%", "")
today_pm_rainy_per = forecast[1].find_all(
    "span", attrs={"class": "text wTable__item"}
)[1].text.replace("%", "")

message = f"""\
本日の最高気温は{today_max_temp}度、最低気温は{today_min_temp}度。
降水確率は午前が{today_am_rainy_per}%、午後が{today_pm_rainy_per}%です。{comment}"""

auth = {"Authorization": "Bearer " + token}
content = {"message": message}
requests.post(url, headers=auth, data=content)

<Response [200]>